# Forest Plot: CHIP vs Synonymous passenger

In [ ]:
## Date: May 30, 2024
# save.image(file="/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240529.rda")

# load("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/data_n_models.passenger_hitchhiker_20240529.rda")

#### Forest plot
##############################
library(data.table) # version 1.14.6
library(meta) # version 6.2-1
library(grid) # version 4.2.2
library(scales) # version 1.2.1
##############################

In [ ]:

##### Forest Plot
dat <- fread("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/passenger_vs_chip.multivariable_glm.20240604.csv", 
header=T) 

# Exposure
dat$Exposure[1] <- "Model 1: Presence of CHIP"
dat$Exposure[2] <- "Model 1: Presence of CHIP"
dat$Exposure[3] <- "Model 1: Presence of CHIP"
dat$Exposure[4] <- "Model 1: Presence of CHIP"
dat$Exposure[5] <- "Model 3: Growth [dVAF>0]"
dat$Exposure[6] <- "Model 2: No. of CHIP clones"
dat$Exposure[7] <- "Model 2: No. of CHIP clones"
dat$Exposure[8] <- "Model 2: No. of CHIP clones"
dat$Exposure[9] <- "Model 2: No. of CHIP clones"

#outcome
dat$Outcome[dat$Outcome=="CHIP_baseline_or_visit05"] <- "i. CHIP at either visit"
dat$Outcome[dat$Outcome=="CH_baseline"] <- "ii. Prevalent CHIP"
dat$Outcome[dat$Outcome=="CH_v05"] <- "iii. CHIP at follow-up"
dat$Outcome[dat$Outcome=="incident_CH"] <- "iv. Incident CHIP"

dat$Outcome[dat$Outcome=="CHIP_expanded_vs_noChange"] <- "Expanded clone"

dat$Outcome[dat$Outcome=="1 CHIP"] <- "1 clone"
dat$Outcome[dat$Outcome=="2 CHIP"] <- "2 clones"
dat$Outcome[dat$Outcome=="3 CHIP"] <- "3 clones"
dat$Outcome[dat$Outcome=="4+ CHIP"] <- "4+ clones"



In [ ]:
###
## Format
dat$Pval <- formatC(x = dat$P, 
                    digits = 1, 
                    format = "E")
# OR
dat$OR <- formatC(round(exp(dat$Beta),2), 
                  digits = 2, format = "f")

dat$lSE <- ( dat$Beta - 1.96 * dat$SE)

dat$uSE <- ( dat$Beta + 1.96 * dat$SE)
# 95% CI
dat$CI95 <- paste0("[",formatC(round(exp( dat$Beta - 1.96 * dat$SE),2), digits = 2, format = "f"),
                   ", ",
                   formatC(round(exp( dat$Beta + 1.96 * dat$SE),2), digits = 2, format = "f"), 
                   "]")



In [ ]:
dat

In [ ]:
table(dat$Outcome)

table(dat$Exposure)

In [ ]:


##
dat$Outcome <- ordered(dat$Outcome, 
                       levels = c("i. CHIP at either visit",
                                  "ii. Prevalent CHIP", 
                                  "iii. CHIP at follow-up", 
                                  "iv. Incident CHIP",
                                   "1 clone", 
                                  "2 clones","3 clones", "4+ clones",
                                  "Expanded clone"))
## Exposures
dat$Exposure <- ordered(dat$Exposure, 
                       levels = c("Model 1: Presence of CHIP",
                                  "Model 2: No. of CHIP clones", 
                                  "Model 3: Growth [dVAF>0]"))



In [ ]:

## Format metagen
dat_nSyno <- metagen(TE = Beta,
                    lower = lSE,
                    upper = uSE,
                    studlab = Outcome,
                    subgroup=Exposure,
                    data=dat,
                    sm="OR")

### 
pdf("/medpop/esp2/mesbah/projects/ch_progression/aric/passenger/Fig6.passenger_count.Forestplot.20240612.pdf", 
    width = 8, height= 8)

forest(x = dat_nSyno, 
       common=F, 
       random=F, 
       hetstat=F, 
       subgroup=k.w>=1, 
       weight.study="same",  
       level=0.95, 
       xlim=c(0.5, 5.5), 
       smlab="Effect of synonymous passenger counts\non CHIP\n", 
       smlab.pos=0, 
       colgap=unit(7, "mm"),
       xlab="Odds Ratio", 
       squaresize=0.6, 
       col.subgroup="black", 
       colgap.left=unit(0.1,"cm"),
       colgap.forest.left="3mm", 
       colgap.forest.right="2mm", 
       leftcols=c("studlab"), 
       leftlabs = c("                     "),
       rightcols=c("OR","CI95","Pval"),
       rightlabs=c("OR","95% CI","P", ""),
       #rightcols=NULL, 
       #rightlabs=NULL,
       col.inside="black", 
       plotwidth=unit(6.5, "cm"), 
       print.subgroup.name=F)

dev.off()

